**This is example of how to trace model with jit and export it to the onnx**

In [2]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.14.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/viktorm/anaconda3/envs/warp39

  added / updated specs:
    - onnx


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libprotobuf-3.20.1         |       h4ff587b_0         2.1 MB
    onnx-1.12.0                |   py39h6736146_0         7.3 MB
    protobuf-3.20.1            |   py39h295c915_0         293 KB
    ------------------------------------------------------------
                                           Total:         9.7 MB

The following NEW packages will be INSTALLED:

  libprotobuf        pkgs/main/linux-64::libprotobuf-3.20.1-h4ff587b_0
  onnx               pkgs/main/linux-64::onnx-1.12.0-py39h6736146_0
  protobuf     

ModuleNotFoundError: No module named 'onnx'

In [ ]:
ray.init(object_store_memory=1024*1024*1000)

In [ ]:
config_name = 'rl_games/configs/ppo_cartpole.yaml'

In [ ]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['params']['config']['full_experiment_name'] = 'cartpole_onnx'
    config['params']['config']['normalize_input'] = True
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

In [ ]:
class ModelWrapper(torch.nn.Module):
    '''
    Main idea is to ignore outputs which we don't need from model
    '''
    def __init__(self, model):
        torch.nn.Module.__init__(self)
        self._model = model
        
        
    def forward(input_dict):
        output_dict = self._model(input_dict)
        
        return output_dict['logits'], output_dict['value'] # may require rnn_states and/or other outputs

In [ ]:
agent = runner.create_player()
agent.restore('runs/cartpole_onnx/nn/cartpole_vel_info.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states,
    'is_train' : False,
}

with torch.no_grad():
    adapter = flatten.TracingAdapter(ModelWrapper(agent.model), inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "cartpole.onnx", verbose=True, input_names=['obs'], output_names=['logits', 'value'])

onnx_model = onnx.load("cartpole.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [ ]:
ort_model = ort.InferenceSession("cartpole.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 4)).astype(np.float32)},
)
print(outputs)

In [ ]:
is_done = False
env = agent.env
obs = env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    action = np.argmax(outputs[0])
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    #plt.imshow(screen)
    #display.display(plt.gcf())    
    #display.clear_output(wait=True)
print(total_reward, num_steps)
#ipythondisplay.clear_output(wait=True)